In [1]:
!pip install xlrd
import pandas as pd


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [2]:
Hosp_sheet = pd.read_excel('datos.xls', sheet_name='Hospitales')

In [3]:
Info_sheet = pd.read_excel('datos.xls', sheet_name='Información Histórica')

In [4]:
GRD_sheet = pd.read_excel('datos.xls', sheet_name='GRD')
GRD_sheet.head()

,ID GRD,Nombre GRD,Costo resolución en Clínica Privada [MM$]
0,GRD1,Coronario,7.917
1,GRD2,Hepático,3.417
2,GRD3,Respiratorio,2.915
3,GRD4,Renal,6.188
4,GRD5,Neurológico,6.782


### Pregunta 1

In [22]:
pacientes_dia_1 = Info_sheet[Info_sheet['Día de llegada'] == 1]

pacientes = Hosp_sheet.merge(pacientes_dia_1, how='inner', left_on='Nombre Establecimiento', right_on='Hospital al que llega')
pacientes[['Nombre Servicio de Salud', 'Día de llegada']].groupby('Nombre Servicio de Salud').count().sort_values('Día de llegada', ascending=False)

,Día de llegada
Nombre Servicio de Salud,
Metropolitano Oriente,36
Metropolitano Occidente,29
Metropolitano Sur,26
Metropolitano Central,25
Metropolitano Norte,16
Metropolitano Sur Oriente,14


### Pregunta 2

In [32]:
GRD_1 = Info_sheet[Info_sheet['GRD'] == 'GRD1']
GRD_7 = Info_sheet[Info_sheet['GRD'] == 'GRD7']
GRD_10 = Info_sheet[Info_sheet['GRD'] == 'GRD10']

concat_hist_df = pd.concat([GRD_1, GRD_7, GRD_10])
concat_hist_df.head()

,Número de llegada,Hospital al que llega,GRD,Día de llegada,Día de salida,Estadia
5,6,Instituto Nacional de Enfermedades Respiratori...,GRD1,1,11,10
7,8,Hospital Dr. Luis Tisné B.,GRD1,1,11,10
8,9,Hospital Del Salvador,GRD1,1,11,10
18,19,Hospital Barros Luco Trudeau,GRD1,1,11,10
19,20,Hospital Dr. Félix Bulnes Cerda,GRD1,1,11,10


In [38]:
met_sur = 'Metropolitano Sur                                 '
pacientes_grd_1_7_10 = Hosp_sheet[Hosp_sheet['Nombre Servicio de Salud'] == met_sur].merge(concat_hist_df, how='inner', left_on='Nombre Establecimiento', right_on='Hospital al que llega')
pacientes_grd_1_7_10[['GRD', 'Nombre Servicio de Salud']].groupby('GRD').count()

,Nombre Servicio de Salud
GRD,
GRD1,1311
GRD10,1269
GRD7,1293


### Pregunta 3

In [58]:

Info_sheet[['Hospital al que llega', 'GRD']].groupby('Hospital al que llega').count()['GRD']

total_x_hospital = pd.DataFrame(Info_sheet[['Hospital al que llega', 'GRD']].groupby('Hospital al que llega').count()['GRD'])

In [61]:
total_x_hospital = total_x_hospital.rename(columns={"GRD": 'total_pacientes'})

In [74]:
total_x_hospital['Hospital'] = total_x_hospital.index.get_level_values('Hospital al que llega')

merge_df_3 = Hosp_sheet.merge(total_x_hospital, how='inner', left_on='Nombre Establecimiento', right_on='Hospital')
merge_df_3[['Nombre Servicio de Salud', 'total_pacientes']].groupby('Nombre Servicio de Salud').median()

,total_pacientes
Nombre Servicio de Salud,
Metropolitano Central,4688.0
Metropolitano Norte,4652.0
Metropolitano Occidente,3265.5
Metropolitano Oriente,4050.0
Metropolitano Sur,1611.0
Metropolitano Sur Oriente,1608.0


### Pregunta 4

In [90]:
last_10 = Info_sheet[Info_sheet['Día de llegada'] >= 415]
elcarmen = last_10[last_10['Hospital al que llega'] == 'Hospital el Carmen']
elcarmen_grd1_10 = elcarmen[elcarmen['GRD'] == 'GRD1']


In [91]:
elcarmen_grd1_10


,Número de llegada,Hospital al que llega,GRD,Día de llegada,Día de salida,Estadia
64725,64726,Hospital el Carmen,GRD1,419,429,10
65419,65420,Hospital el Carmen,GRD1,423,433,10


In [92]:
nombre_est = 'Nombre Establecimiento'
merge_df_4 = Hosp_sheet.merge(elcarmen_grd1_10, how='inner', left_on='Nombre Establecimiento', right_on='Hospital al que llega')

In [101]:
costo_4_grd1 = merge_df_4['Estadia'] * merge_df_4['Costo diario GRD1 [MM$]']

In [102]:
costo_4_grd1.sum()

8.88

### Pregunta 5

In [105]:
last_10 = Info_sheet[Info_sheet['Día de llegada'] >= 415]
elcarmen = last_10[last_10['Hospital al que llega'] == 'Hospital el Carmen']

merge_df_5 = Hosp_sheet.merge(elcarmen, how='inner', left_on='Nombre Establecimiento', right_on='Hospital al que llega')
merge_df_5.head()

,Nombre Servicio de Salud,Nombre Establecimiento,Área Cuidados Intensivos Adultos,Área Cuidados Intermedios Adulto,Total Camas Críticas,Máxima cantidad de camas a aumetar,Costo de instalar una cama [MM$],Costo de operación anual de una cama [MM$],Costo diario GRD1 [MM$],Costo diario GRD2 [MM$],...,Costo diario GRD7 [MM$],Costo diario GRD8 [MM$],Costo diario GRD9 [MM$],Costo diario GRD10 [MM$],Número de llegada,Hospital al que llega,GRD,Día de llegada,Día de salida,Estadia
0,Metropolitano Central ...,Hospital el Carmen,6,12,18,4,1368,49.7,0.444,0.439,...,0.302,0.414,0.494,0.425,63993,Hospital el Carmen,GRD4,415,421,6
1,Metropolitano Central ...,Hospital el Carmen,6,12,18,4,1368,49.7,0.444,0.439,...,0.302,0.414,0.494,0.425,64150,Hospital el Carmen,GRD8,416,420,4
2,Metropolitano Central ...,Hospital el Carmen,6,12,18,4,1368,49.7,0.444,0.439,...,0.302,0.414,0.494,0.425,64153,Hospital el Carmen,GRD6,416,421,5
3,Metropolitano Central ...,Hospital el Carmen,6,12,18,4,1368,49.7,0.444,0.439,...,0.302,0.414,0.494,0.425,64174,Hospital el Carmen,GRD10,416,426,10
4,Metropolitano Central ...,Hospital el Carmen,6,12,18,4,1368,49.7,0.444,0.439,...,0.302,0.414,0.494,0.425,64374,Hospital el Carmen,GRD2,417,421,4


In [111]:
costo_5 = 0.0

for _, row in merge_df_5.iterrows():
    grd = row['GRD']
    estadia = row['Estadia']
    costo_diario = row[f'Costo diario {grd} [MM$]']
    print(f'grd: {grd}, estadia: {estadia}, Costo diario {grd} [MM$]: {costo_diario}')
    costo_5 += estadia * costo_diario

print(f'\n\n Costo final: {costo_5}')

grd: GRD4, estadia: 6, Costo diario GRD4 [MM$]: 0.569
grd: GRD8, estadia: 4, Costo diario GRD8 [MM$]: 0.414
grd: GRD6, estadia: 5, Costo diario GRD6 [MM$]: 0.416
grd: GRD10, estadia: 10, Costo diario GRD10 [MM$]: 0.425
grd: GRD2, estadia: 4, Costo diario GRD2 [MM$]: 0.439
grd: GRD3, estadia: 3, Costo diario GRD3 [MM$]: 0.386
grd: GRD6, estadia: 5, Costo diario GRD6 [MM$]: 0.416
grd: GRD5, estadia: 4, Costo diario GRD5 [MM$]: 0.404
grd: GRD3, estadia: 3, Costo diario GRD3 [MM$]: 0.386
grd: GRD3, estadia: 3, Costo diario GRD3 [MM$]: 0.386
grd: GRD2, estadia: 4, Costo diario GRD2 [MM$]: 0.439
grd: GRD3, estadia: 3, Costo diario GRD3 [MM$]: 0.386
grd: GRD9, estadia: 5, Costo diario GRD9 [MM$]: 0.494
grd: GRD9, estadia: 5, Costo diario GRD9 [MM$]: 0.494
grd: GRD1, estadia: 10, Costo diario GRD1 [MM$]: 0.444
grd: GRD10, estadia: 10, Costo diario GRD10 [MM$]: 0.425
grd: GRD8, estadia: 4, Costo diario GRD8 [MM$]: 0.414
grd: GRD6, estadia: 5, Costo diario GRD6 [MM$]: 0.416
grd: GRD9, estadia: 5

### Pregunta 6

In [39]:
costo_diario_grd_servicio = Hosp_sheet[['Nombre Servicio de Salud',
                                        'Costo diario GRD1 [MM$]',
                                        'Costo diario GRD2 [MM$]',
                                        'Costo diario GRD3 [MM$]',
                                        'Costo diario GRD4 [MM$]',
                                        'Costo diario GRD5 [MM$]',
                                        'Costo diario GRD6 [MM$]',
                                        'Costo diario GRD7 [MM$]',
                                        'Costo diario GRD8 [MM$]',
                                        'Costo diario GRD9 [MM$]',
                                        'Costo diario GRD10 [MM$]']].groupby('Nombre Servicio de Salud').mean()
costo_diario_grd_servicio['Servicio'] = costo_diario_grd_servicio.index.get_level_values('Nombre Servicio de Salud')

costo_diario_grd_servicio.head()

,Costo diario GRD1 [MM$],Costo diario GRD2 [MM$],Costo diario GRD3 [MM$],Costo diario GRD4 [MM$],Costo diario GRD5 [MM$],Costo diario GRD6 [MM$],Costo diario GRD7 [MM$],Costo diario GRD8 [MM$],Costo diario GRD9 [MM$],Costo diario GRD10 [MM$],Servicio
Nombre Servicio de Salud,,,,,,,,,,,
Metropolitano Central,0.473667,0.43700,0.434667,0.49800,0.38700,0.462333,0.4450,0.435667,0.381333,0.522333,Metropolitano Central ...
Metropolitano Norte,0.593000,0.50900,0.367000,0.49200,0.52400,0.405000,0.5710,0.474000,0.498000,0.305000,Metropolitano Norte ...
Metropolitano Occidente,0.490500,0.42125,0.459000,0.38875,0.51025,0.412250,0.4560,0.369250,0.489750,0.365500,Metropolitano Occidente ...
Metropolitano Oriente,0.426000,0.44050,0.448500,0.44375,0.52000,0.396250,0.4030,0.470500,0.423500,0.430750,Metropolitano Oriente ...
Metropolitano Sur,0.515200,0.44680,0.542800,0.43900,0.51900,0.403800,0.3892,0.515200,0.489000,0.383600,Metropolitano Sur ...


In [40]:
met_central = 'Metropolitano Central                             '
costos_met_central = costo_diario_grd_servicio[costo_diario_grd_servicio['Servicio'] == met_central]

In [41]:
servicio_en_info = Info_sheet[Info_sheet['Día de llegada'] >= 415].merge(Hosp_sheet, how='inner', left_on='Hospital al que llega', right_on='Nombre Establecimiento')
servicio_en_info = servicio_en_info[['Hospital al que llega', 'Nombre Servicio de Salud', 'GRD', 'Día de llegada', 'Día de salida', 'Estadia']]
servicio_en_info.head()

,Hospital al que llega,Nombre Servicio de Salud,GRD,Día de llegada,Día de salida,Estadia
0,Complejo Hospitalario Dr. Sótero del Río,Metropolitano Sur Oriente ...,GRD2,415,419,4
1,Complejo Hospitalario Dr. Sótero del Río,Metropolitano Sur Oriente ...,GRD8,415,419,4
2,Complejo Hospitalario Dr. Sótero del Río,Metropolitano Sur Oriente ...,GRD2,415,419,4
3,Complejo Hospitalario Dr. Sótero del Río,Metropolitano Sur Oriente ...,GRD9,415,420,5
4,Complejo Hospitalario Dr. Sótero del Río,Metropolitano Sur Oriente ...,GRD3,415,418,3


In [42]:
merge_df_6 = costos_met_central.merge(servicio_en_info, how='inner', left_on='Servicio', right_on='Nombre Servicio de Salud')
merge_df_6.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 268 entries, 0 to 267
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Costo diario GRD1 [MM$]   268 non-null    float64
 1   Costo diario GRD2 [MM$]   268 non-null    float64
 2   Costo diario GRD3 [MM$]   268 non-null    float64
 3   Costo diario GRD4 [MM$]   268 non-null    float64
 4   Costo diario GRD5 [MM$]   268 non-null    float64
 5   Costo diario GRD6 [MM$]   268 non-null    float64
 6   Costo diario GRD7 [MM$]   268 non-null    float64
 7   Costo diario GRD8 [MM$]   268 non-null    float64
 8   Costo diario GRD9 [MM$]   268 non-null    float64
 9   Costo diario GRD10 [MM$]  268 non-null    float64
 10  Servicio                  268 non-null    object 
 11  Hospital al que llega     268 non-null    object 
 12  Nombre Servicio de Salud  268 non-null    object 
 13  GRD                       268 non-null    object 
 14  Día de lle

In [43]:
costo_6 = 0.0

for _, row in merge_df_6.iterrows():
    grd = row['GRD']
    estadia = row['Estadia']
    costo_diario = row[f'Costo diario {grd} [MM$]']
    print(f'grd: {grd}, estadia: {estadia}, Costo diario {grd} [MM$]: {costo_diario}')
    costo_6 += estadia * costo_diario

print(f'\n\n Costo final: {costo_6}')

grd: GRD8, estadia: 4, Costo diario GRD8 [MM$]: 0.43566666666666665
grd: GRD6, estadia: 5, Costo diario GRD6 [MM$]: 0.4623333333333333
grd: GRD4, estadia: 6, Costo diario GRD4 [MM$]: 0.498
grd: GRD7, estadia: 4, Costo diario GRD7 [MM$]: 0.445
grd: GRD9, estadia: 5, Costo diario GRD9 [MM$]: 0.38133333333333336
grd: GRD8, estadia: 4, Costo diario GRD8 [MM$]: 0.43566666666666665
grd: GRD2, estadia: 4, Costo diario GRD2 [MM$]: 0.437
grd: GRD2, estadia: 4, Costo diario GRD2 [MM$]: 0.437
grd: GRD1, estadia: 10, Costo diario GRD1 [MM$]: 0.4736666666666667
grd: GRD7, estadia: 4, Costo diario GRD7 [MM$]: 0.445
grd: GRD7, estadia: 4, Costo diario GRD7 [MM$]: 0.445
grd: GRD4, estadia: 6, Costo diario GRD4 [MM$]: 0.498
grd: GRD9, estadia: 5, Costo diario GRD9 [MM$]: 0.38133333333333336
grd: GRD1, estadia: 10, Costo diario GRD1 [MM$]: 0.4736666666666667
grd: GRD2, estadia: 4, Costo diario GRD2 [MM$]: 0.437
grd: GRD10, estadia: 10, Costo diario GRD10 [MM$]: 0.5223333333333333
grd: GRD2, estadia: 4, 

### Pregunta 7

In [61]:
servicios = costo_diario_grd_servicio['Servicio'].to_list()

In [63]:
for servicio in servicios:
    costos_diario_df = costo_diario_grd_servicio[costo_diario_grd_servicio['Servicio'] == servicio]

    met_df = costos_diario_df.merge(servicio_en_info, how='inner', left_on='Servicio', right_on='Nombre Servicio de Salud')

    costo_total = 0.0

    for _, row in met_df.iterrows():
        grd = row['GRD']
        estadia = row['Estadia']
        costo_diario = row[f'Costo diario {grd} [MM$]']
        costo_total += estadia * costo_diario

    print(f'Costo {servicio}: {costo_total}')

Costo Metropolitano Central                             : 683.2716666666664
Costo Metropolitano Norte                               : 280.5060000000002
Costo Metropolitano Occidente                           : 610.7789999999984
Costo Metropolitano Oriente                             : 891.3069999999998
Costo Metropolitano Sur                                 : 788.9134000000017
Costo Metropolitano Sur Oriente                         : 597.3847500000002


### Pregunta 8

In [64]:
891.3069999999998 - 280.5060000000002

610.8009999999996

### Pregunta 9

In [75]:
df_9 = Hosp_sheet[['Nombre Servicio de Salud', 'Costo de instalar una cama [MM$]']].groupby('Nombre Servicio de Salud').min() * 8
df_9.sort_values(by='Costo de instalar una cama [MM$]', ascending=False).head(10)

,Costo de instalar una cama [MM$]
Nombre Servicio de Salud,
Metropolitano Norte,11256
Metropolitano Central,10048
Metropolitano Occidente,8528
Metropolitano Oriente,8448
Metropolitano Sur,8408
Metropolitano Sur Oriente,8224


### Pregunta 10

In [76]:
hospitales = Hosp_sheet.loc[Hosp_sheet["Área Cuidados Intensivos Adultos"] < Hosp_sheet["Área Cuidados Intermedios Adulto"]]

hospitales[["Nombre Establecimiento","Área Cuidados Intensivos Adultos","Área Cuidados Intermedios Adulto"]] 

,Nombre Establecimiento,Área Cuidados Intensivos Adultos,Área Cuidados Intermedios Adulto
0,Complejo Hospitalario San José,11,33
1,Hospital Adalberto Steeger (Talagante),6,11
2,Hospital Dr. Félix Bulnes Cerda,3,6
3,Hospital San Juan de Dios,29,47
5,Hospital Clínico San Borja-Arriarán,17,26
7,Hospital el Carmen,6,12
8,Hospital Del Salvador,22,24
9,Hospital Dr. Luis Tisné B.,8,16
11,Instituto Nacional de Enfermedades Respiratori...,15,29
12,Hospital Barros Luco Trudeau,31,43


In [77]:

hospitales = Hosp_sheet[Hosp_sheet["Área Cuidados Intensivos Adultos"] < Hosp_sheet["Área Cuidados Intermedios Adulto"]]

hospitales[["Nombre Establecimiento","Área Cuidados Intensivos Adultos","Área Cuidados Intermedios Adulto"]]

,Nombre Establecimiento,Área Cuidados Intensivos Adultos,Área Cuidados Intermedios Adulto
0,Complejo Hospitalario San José,11,33
1,Hospital Adalberto Steeger (Talagante),6,11
2,Hospital Dr. Félix Bulnes Cerda,3,6
3,Hospital San Juan de Dios,29,47
5,Hospital Clínico San Borja-Arriarán,17,26
7,Hospital el Carmen,6,12
8,Hospital Del Salvador,22,24
9,Hospital Dr. Luis Tisné B.,8,16
11,Instituto Nacional de Enfermedades Respiratori...,15,29
12,Hospital Barros Luco Trudeau,31,43
